In [1]:
from pyeasyga import pyeasyga
import random

In [2]:
# setup data
f = open('8.txt')
data = []
lines=f.read()
lines=lines.split('\n')
maxWeight=0
maxVolume=0
for line in lines:
    lines = line.split(' ')
    if(len(lines)==2):
        maxWeight=int(lines[0])
        maxVolume=int(lines[1])
        d= (float(lines[0]),float(lines[1]))
    else:
        d= (float(lines[0]),float(lines[1]),float(lines[2]))
    
    data.append(d)
data=data[1:]
print(data)
print(maxWeight)
print(maxVolume) 

[(1094.0, 0.4, 339.0), (810.0, 1.1, 258.0), (1698.0, 0.5, 106.0), (1283.0, 0.5, 189.0), (924.0, 0.7, 356.0), (456.0, 0.8, 103.0), (867.0, 0.6, 195.0), (1350.0, 0.7, 123.0), (1107.0, 1.1, 152.0), (674.0, 1.0, 329.0), (543.0, 0.8, 335.0), (120.0, 1.1, 323.0), (1279.0, 0.8, 349.0), (1083.0, 0.9, 352.0), (572.0, 1.2, 335.0), (209.0, 0.7, 351.0), (471.0, 0.5, 198.0), (1097.0, 0.9, 169.0), (1286.0, 0.5, 353.0), (366.0, 1.1, 136.0), (1240.0, 0.6, 304.0), (168.0, 1.0, 176.0), (1114.0, 0.5, 387.0), (852.0, 0.9, 205.0), (267.0, 0.5, 140.0), (1390.0, 0.5, 195.0), (1207.0, 0.9, 241.0), (468.0, 0.5, 343.0), (601.0, 0.9, 285.0), (621.0, 1.0, 124.0)]
13000
12


In [3]:
print(len(data) )


30


In [4]:
## создание генома через жааааааадный алгоритм (беру от жизни все!!) 
def createGenome(data,population):
    sizeGenome=len(data)
    anger=[]
    for item in data:
        anger.append(item[2]/item[0])
    sumGluttony=[]
    copyDataPride=data
    for item1 in data:
        maxPr=0
        indMax=-1
        for i in range(0,len(anger)):
            if(maxPr<anger[i]):
                maxPr=anger[i]
                indMax=i
        sumGluttony.append(int(indMax))
        anger[indMax]=-1
    genomesind=[]
    rezervcopy=sumGluttony.copy()
    for individual in range(0,population):
        genome=[0 ]* sizeGenome
        ran=random.randint(0,sizeGenome-1) 
        weight = data[ran][0] 
        volume = data[ran][1]
        genome[ran]=0
        sumGluttony=rezervcopy.copy() 
        del sumGluttony[ran]
        for item in sumGluttony: 
            weight += data[item][0]
            volume += data[item][1]
            if weight > maxWeight or volume > maxVolume:
                break
            else:
                genome[item]=1
        genomesind.append(genome)
    return genomesind

In [5]:
# define a fitness function
def fitness(individual, data):
    weight, volume, price = 0, 0, 0
    for (selected, item) in zip(individual, data):
        if selected:
            weight += item[0]
            volume += item[1]
            price += item[2]
    if weight > maxWeight or volume > maxVolume:
        price = 0
    return price 

In [6]:
##Посчитаем сходимость через фитнес функцию и выберем родителя
def choiceParent(data,genomesind):
    convergence=[]
    i=0
    for genome in genomesind:
        convergence.append((fitness(genome,data),i))
        i=i+1
    convergence=sorted(convergence, key=lambda convergence: convergence[0] ,reverse=True)
    Parents=[]
    for i in range(0,int(len(genomesind)/2)):
        sumRuletca=0
        FirstParent=0
        SecondParent=0
        for item in convergence:
            sumRuletca+=item[0]
        ran=random.randint(0,sumRuletca-1)
        choice=0
        j=0
        for item in convergence:
            choice+=item[0]
            if(ran<=choice):
                FirstParent=item[1]
                del convergence[j]
                break
            j+=1
        sumRuletca=0    
        for item in convergence:
            sumRuletca+=item[0]
        ran=random.randint(0,sumRuletca)
        choice=0
        j=0
        for item in convergence:
            choice+=item[0]
            if(ran<=choice):
                SecondParent=item[1]
                del convergence[j]
                break
            j+=1
        Parents.append((FirstParent,SecondParent))

    return Parents

In [7]:
##функция скрещивания
def crossing_over(Parent,data,genomesind,n):
    new_gen=[]
    points=[]
    for i in range(0,n):
        fl=True
        while(fl):
            fl=False
            ran=random.randint(0, len(data)-1)
        
            for point in points:
                if(point==ran):
                    fl=True
        points.append(ran)
    points.sort()
    for family in Parent:
        firstChild=genomesind[family[0]][:points[0]]
        secondChild=genomesind[family[1]][:points[0]]
        for i in range(0,n):
            if(i%2==1):
                if(i==n-1):
                    temp=genomesind[family[0]][points[i]:]
                    firstChild+=genomesind[family[1]][points[i]:]
                    secondChild+=temp
                else:
                    temp=genomesind[family[0]][points[i]:points[i+1]]
                    firstChild+=genomesind[family[1]][points[i]:points[i+1]]
                    secondChild+=temp
            else:
                if(i==n-1):
                    firstChild+=genomesind[family[0]][points[i]:]
                    secondChild+=genomesind[family[1]][points[i]:]
                else: 
                    firstChild+=genomesind[family[0]][points[i]:points[i+1]]
                    secondChild+=genomesind[family[1]][points[i]:points[i+1]]
        new_gen.append(firstChild)
        new_gen.append(secondChild)
    return new_gen


In [8]:
def chanceMutation(genomesind,percent, count):
    new_genomesind=[]
    leng=int(percent*len(genomesind))
    for i in range(0,leng):
        ran=random.randint(0, len(genomesind)-1)
        changeGen=genomesind[ran]
        points=[]
        for j in range(0,count):
                fl=True
                while(fl):
                    fl=False
                    ran=random.randint(0, len(data)-1)

                    for point in points:
                        if(point==ran):
                            fl=True
                points.append(ran)
        for point in points:
            
            if(changeGen[point]==0):
                changeGen[point]=1
            else:
                changeGen[point]=0
        del genomesind[ran]
        new_genomesind.append(changeGen)
    return new_genomesind+genomesind
            

In [9]:
 def replacement(data,old_genomesind,new__genomesind,percent):
    i=0
    OldFit=[]
    for genome in old_genomesind:
        OldFit.append((fitness(genome,data),i))
        i=i+1
    OldFit=sorted(OldFit, key=lambda OldFit: OldFit[0] ,reverse=True)
    i=0
    NewFit=[]
    for genome in new__genomesind:
        NewFit.append((fitness(genome,data),i))
        i=i+1
    NewFit=sorted(NewFit, key=lambda NewFit: NewFit[0] ,reverse=True)
    new_generation=[]
    for i in range(0,len(OldFit)-int(percent*len(old_genomesind))):
        new_generation.append(old_genomesind[OldFit[i][1]])
    for i in range(0,int(percent*len(new__genomesind))):
        new_generation.append(new__genomesind[NewFit[i][1]])
    return new_generation
    

In [10]:
def minPrice(data):
    minPr=9999999
    for item in data:
        if (minPr>item[2]):
            minPr=item[2]
    return minPr

In [11]:
def result_evaluation(genomesind,data, itaration,maxIter,best,minPr):
    if (itaration==maxIter):
        return True
    if (best_of_best(genomesind,data)-best<minPr and best_of_best(genomesind,data)-best>0):
        return True
    return False

In [12]:
def best_of_best(genomesind,data):
    ##i=0
    ##resul=[]
    summ=0
    for genome in genomesind:
        summ+=fitness(genome,data)
    return summ
        ##resul.append((fitness(genome,data),i))
        ##i=i+1
        ##resul=sorted(resul, key=lambda resul: resul[0] ,reverse=True)
    #return resul[0][0]

In [13]:
## 0. Кодирование
## Логика выбора генома проста - берет себе в рюкзак конкретную вешь это 1
## если не берет, то 0
## таким образом n размерный геном, где n - количество вещей
population=200
itaration=0
maxIter=400
minPr=minPrice(data)
## 1. Начальная популяция
## 1.2 Жааааааадный алгоритм (беру от жизни все!!) 
genomesind=createGenome(data,population)
while True:
    itaration+=1
    best=best_of_best(genomesind,data)
## 2. Отбор особей для скрещивания
## 2.1 выбор каждой особи пропорционально приспособленности (рулетка)
    Parent=choiceParent(data,genomesind)
## 3. Скрещивание (кроссинговер) между выбранными особями
## 3.1 многоточечный с 3мя точками 
    new_gen=crossing_over(Parent,data,genomesind,3)  
## 4. Мутация
## 4.2 случайное изменение 3х битов у 5% особей
    genomesind=chanceMutation(genomesind,0.05, 3)
    new_gen=chanceMutation(new_gen,0.05, 3)
## 5. Формирование новой популяции
## 5.1 замена 20% худших особей из предыдущего поколения на лучших потомков
    genomesind = replacement(data,genomesind,new_gen,0.2)
## 6. Оценка результата
    res=result_evaluation(genomesind,data, itaration,maxIter,best,minPr)
    if(res):
        break
        
i=0
resul=[]
for genome in genomesind:
    resul.append((fitness(genome,data),i))
    i=i+1
resul=sorted(resul, key=lambda resul: resul[0] ,reverse=True)
print(resul[0][0])
resul=(resul[0][0],genomesind[resul[0][1]])
print(itaration)

5134.0
38


In [14]:
for i in range(0,len(resul[1])):
    if(resul[1][i]==1):
        print(data[i])

(1094.0, 0.4, 339.0)
(924.0, 0.7, 356.0)
(867.0, 0.6, 195.0)
(674.0, 1.0, 329.0)
(543.0, 0.8, 335.0)
(120.0, 1.1, 323.0)
(1279.0, 0.8, 349.0)
(1083.0, 0.9, 352.0)
(572.0, 1.2, 335.0)
(209.0, 0.7, 351.0)
(471.0, 0.5, 198.0)
(1286.0, 0.5, 353.0)
(1240.0, 0.6, 304.0)
(1114.0, 0.5, 387.0)
(468.0, 0.5, 343.0)
(601.0, 0.9, 285.0)
